In [17]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [18]:
df_s = pd.read_csv('ship.csv', low_memory=False).rename(columns={'id': 'ship_id'})
df_test = pd.read_csv('absent_ships2.csv').rename(columns={'correct.ship_index/new ship_id': 'ship_index'}).drop(columns=['ship_id_old', 'wrong.ship_index', 'voyage_index', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8'])
df_s1 = pd.read_csv('ship_by_id2.csv', low_memory=False).rename(columns={'id': 'ship_id'}).drop(columns=['Unnamed: 9', '328 ship names'])

In [19]:
df_test['ship_index'] = df_test['ship_index'].str.replace(" ", "", regex=False)

In [20]:
# for index, row in df_test.iterrows():
#     if '/' in row['ship_index']:
#         target = 0
#         if row['ship_index'].split('/')[0] == '':
#             target = 1
#         df_test.at[index, 'ship_index'] = row['ship_index'].split('/')[target].strip()

In [21]:
df_s['ship_name'] = df_s['ship_name'].str.strip().str.title()
df_s1['ship_name'] = df_s1['ship_name'].str.strip().str.title()
df_s['ship_code'] = df_s['ship_code'].str.strip().str.upper()
df_s1['ship_code'] = df_s1['ship_code'].str.strip().str.upper()
df_s['alt_ship_name'] = np.nan
df_s1['ship_index'] = np.nan

In [5]:
for index, row in df_s1.iterrows():
    match = df_s[df_s['ship_code'] == row['ship_code']]
    if not match.empty:
        df_s1.at[index, 'ship_index'] = match['ship_index'].values[0]

In [6]:
# for index, row in df_s1.iterrows():
#     if row['ship_code'] not in df_s['ship_code'].values:
#         new_row = row.to_dict()
#         new_row['ship_index'] = None

#         df_s = pd.concat([df_s, pd.DataFrame([new_row])], ignore_index=True)

In [7]:
# df_merged = pd.merge(
#     df_s,
#     df_s1,
#     on=['ship_code', 'ship_name'],
#     how='outer'
# )
# for i in list(df_s.columns):
#     if i != 'ship_name' and i != 'ship_code' and i != 'ship_index':
#         df_merged[i] = df_merged[i+'_x'].combine_first(df_merged[i+'_y'])
#         df_merged.drop(columns=[i+'_x', i+'_y'], inplace=True)
# df_merged

In [8]:
for index, row in tqdm(df_s1.iterrows()):
    if pd.isna(row['ship_index']) and len(df_test[df_test['ship_code'] == row['ship_code']]) > 0:
        df_s1.at[index, 'ship_index'] = df_test[df_test['ship_code'] == row['ship_code']]['ship_index'].iloc[0]

590it [00:00, 2610.63it/s]


In [9]:
# find duplicated ship_codes
codes = list(df_s['ship_code'])
temp, res, dc = [], [], {}
for i in codes:
    if i not in temp:
        temp.append(i)
    else:
        res.append(i)
    if len(df_s[df_s['ship_code']==i]) > 1 and i != '555':
        dc[i] = list(df_s[df_s['ship_code']==i]['ship_index'])

In [10]:
df_s1['ship_index'] = pd.to_numeric(df_s1['ship_index'], errors='coerce').astype('Int64')
for code, (correct, wrong) in dc.items():
    mask = (df_s1['ship_code'] == code) & (df_s1['ship_index'] == wrong)
    df_s1.loc[mask, 'ship_index'] = correct

In [11]:
a = list(df_s1['ship_index'])
cnt = []
for i in a:
    cnt.append(list(df_s1[df_s1['ship_index']==i]['ship_index']))
res = [i[0] for i in cnt if len(i) > 1]
set(res)

{1,
 91,
 112,
 132,
 234,
 255,
 282,
 283,
 288,
 298,
 301,
 402,
 432,
 439,
 440,
 443,
 478,
 499,
 500,
 501,
 510,
 521,
 531,
 541,
 572}

In [33]:
df_s1[df_s1['ship_index'].isin(res)].sort_values(by='ship_index')[['ship_name', 'ship_code', 'ship_index']].to_csv('conflicting_ship_indices.csv')

In [76]:
df_s1.to_csv('latest_ship_table.csv')

In [61]:
df_r = pd.read_csv('record_corrected2.csv', low_memory=False).drop('Unnamed: 29', axis=1).rename(columns={'id': 'record_id'})

In [62]:
for index, row in tqdm(df_r.iterrows()):
    if '/' in row['ship_index']:
        target = 0
        if row['ship_index'].split('/')[0] == '':
            target = 1
        df_r.at[index, 'ship_index'] = row['ship_index'].split('/')[target].strip()

215663it [00:31, 6893.85it/s] 


In [63]:
df_r['ship_index'] = pd.to_numeric(df_r['ship_index'], errors='coerce').astype('Int64')
df_r['voyage_index'] = pd.to_numeric(df_r['voyage_index'], errors='coerce').astype('Int64')
df_r['function_index'] = pd.to_numeric(df_r['function_index'], errors='coerce').astype('Int64')
df_r['origin_index'] = pd.to_numeric(df_r['origin_index'], errors='coerce').astype('Int64')
df_r['person_index'] = pd.to_numeric(df_r['person_index'], errors='coerce').astype('Int64')

In [64]:
df_r[df_r['ship_index']==32]

,record_id,file_name,n,age,build,complexion,conscription,wage,remarks,f_n,...,disemb_date,disemb_loc,disemb_class,days_on_voyage,earnings,person_index,origin_index,function_index,ship_index,voyage_index
11344,1949,Louis et al. - BOEDEC Guillaume BOIZARD Étienn...,1,NaN,NaN,NaN,NaN,75,a fait la campagne,NaN,...,NaN,<Disemb_Loc>,301.0,NaN,NaN,1923,0,0,32,76
11345,1950,Louis et al. - BOEDEC Guillaume BOIZARD Étienn...,2,NaN,NaN,NaN,NaN,24,a fait la campagne,NaN,...,NaN,<Disemb_Loc>,301.0,NaN,NaN,1924,0,1,32,76
11346,1951,Louis et al. - BOEDEC Guillaume BOIZARD Étienn...,3,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,...,NaN,<Disemb_Loc>,301.0,NaN,NaN,1925,1,2,32,76
11347,1952,Louis et al. - BOEDEC Guillaume BOIZARD Étienn...,4,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,...,NaN,<Disemb_Loc>,301.0,NaN,NaN,1926,2,2,32,76
11348,1953,Louis et al. - BOEDEC Guillaume BOIZARD Étienn...,5,NaN,NaN,NaN,NaN,18,embarqué à l'armement \n débarqué le 10/08/1723,NaN,...,NaN,NaN,NaN,NaN,NaN,1927,11,2,32,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209924,25836,ROLE2P17-II.34_1875,20,27.0,m,ch,NaN,22,embarqué à l'armement \n débarqué à ?,735-31,...,NaN,NaN,NaN,NaN,NaN,166377,134,2,32,1176
209925,25862,ROLE2P17-II.34_1875,46,NaN,NaN,NaN,NaN,NaN,embarqué à l'armement \n débarqué à ? --- à la...,NaN,...,NaN,NaN,NaN,NaN,NaN,166378,280,23,32,1176
209926,25863,ROLE2P17-II.34_1875,47,NaN,NaN,NaN,NaN,NaN,embarqué à l'armement \n débarqué à ? --- à la...,NaN,...,NaN,NaN,NaN,NaN,NaN,166379,71,23,32,1176
209927,25869,ROLE2P17-II.34_1875,53,NaN,NaN,NaN,NaN,sans solde,embarqué à l'armement \n débarqué à ? --- noir...,NaN,...,NaN,NaN,NaN,NaN,NaN,166380,0,2,32,1176


In [65]:
for code, (correct, wrong) in tqdm(dc.items()):
    df_r.loc[(df_r['ship_index'] == wrong) & (df_r['ship_code']== code), 'ship_index'] = correct

  0%|          | 0/10 [00:00<?, ?it/s]


KeyError: 'ship_code'

In [ ]:
df_r.to_csv('record_slash_free.csv')

In [48]:
df_r[df_r['ship_index']==0]

,record_id,file_name,n,age,build,complexion,conscription,wage,remarks,f_n,...,disemb_date,disemb_loc,disemb_class,days_on_voyage,earnings,person_index,origin_index,function_index,ship_index,voyage_index
0,35033,ROLE2P22-I.12_1911,2,NaN,NaN,NaN,NaN,120,a fait la campagne --- lieutenant de vaisseau,NaN,...,04/06/1726,<Disemb_Loc>,301.0,584.0,<Earnings>,30229,4,32,0,1325
1,35034,ROLE2P22-I.12_1911,3,NaN,NaN,NaN,NaN,90,a fait la campagne,NaN,...,04/06/1726,<Disemb_Loc>,301.0,584.0,<Earnings>,30230,37,134,0,1325
2,35035,ROLE2P22-I.12_1911,4,NaN,NaN,NaN,NaN,60,a fait la campagne,NaN,...,04/06/1726,<Disemb_Loc>,301.0,584.0,<Earnings>,30231,37,42,0,1325
3,35036,ROLE2P22-I.12_1911,5,NaN,NaN,NaN,NaN,20,a fait la campagne,NaN,...,04/06/1726,<Disemb_Loc>,301.0,584.0,<Earnings>,30232,876,135,0,1325
4,35037,ROLE2P22-I.12_1911,6,NaN,NaN,NaN,NaN,50,resté à terre à Lorient,NaN,...,NaN,NaN,NaN,NaN,NaN,30233,0,6,0,1325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,38733,ROLE2P23-I.7_1802,163,NaN,NaN,NaN,NaN,?,embarqué à Pondichéry le 20/01/1728 \n débarqu...,NaN,...,20/01/1728,NaN,NaN,NaN,NaN,167752,0,27,0,1419
310,38735,ROLE2P23-I.7_1802,165,NaN,NaN,NaN,NaN,?,embarqué à Pondichéry le 20/01/1728 \n débarqu...,NaN,...,20/01/1728,NaN,NaN,NaN,NaN,167753,0,27,0,1419
311,38750,ROLE2P23-I.7_1802,180,NaN,NaN,NaN,NaN,NaN,embarqué à l'île Bourbon le 01/04/1728 \n déba...,NaN,...,11/08/1728,NaN,301.0,NaN,NaN,167754,0,239,0,1419
312,38751,ROLE2P23-I.7_1802,181,NaN,NaN,NaN,NaN,NaN,embarqué à l'île Bourbon le 01/04/1728 \n déba...,NaN,...,11/08/1728,NaN,301.0,NaN,NaN,167755,0,27,0,1419


In [41]:
# df_merged['ship_index'] = df_merged['ship_index'].astype(int)
df_merged[df_merged['ship_index'].isna()]

,ship_name,ship_index,ship_code,alt_ship_name,ship_id,ship_built_date,ship_built_place,ship_type,tonnage,guns
596,Penthièvre,NaN,PEN_B,Paix,434.0,NaN,NaN,NaN,NaN,NaN


In [58]:
df_s1[df_s1['ship_code']=='PEN_B']

,ship_id,ship_name,alt_ship_name,ship_built_date,ship_built_place,ship_type,tonnage,guns,ship_code
434,434,Penthièvre,Paix,NaN,NaN,NaN,NaN,NaN,PEN_B


In [57]:
df_s[df_s['ship_code'] == 'PEN_B'].iloc[0]

ship_id               177
ship_name            Paix
ship_built_date       NaN
ship_built_place      NaN
ship_type             NaN
tonnage               NaN
guns                  NaN
ship_index            176
ship_code           PEN_B
Name: 176, dtype: object